# Al momento de hacer el alineamiento estan tomando los carbonos en lugar de carbonos alfa!!!

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
sys.path.append('../math_tricks/')
import funciones_CLICK as fc
from scipy.spatial.distance import pdist, squareform
import multiprocessing
from functools import partial
import warnings
warnings.simplefilter("ignore")

%matplotlib inline

In [14]:
df_rmsd = pd.read_pickle('../df_rmsd.pkl')

new_df_cliques1= pd.read_pickle('../clique1.pkl')
new_df_cliques2= pd.read_pickle('../clique2.pkl')

df_atoms1= pd.read_pickle('../clique1_df_atributos.pkl')
df_atoms2= pd.read_pickle('../clique2_df_atributos.pkl')

In [16]:
df_new_df_clique1 = new_df_cliques1.values
df_new_df_clique2 = new_df_cliques2.values

df_rmsd_1 = df_rmsd.values

def R_ij(i,j):
    valor = sum([vecs_center_1[pos_res[0], i] * vecs_center_2[pos_res[1], j] for pos_res in parejas])
    return valor

def matrix_R():

    """cliques a comparar: i,j
    desde aqui se itera sobre cada i y hay que variar los vectores
    coordenada
    Regresa la matriz gigante (matriz simetrica del articulo)"""
    # primer renglon
    R11R22R33 = (R_ij(0, 0) + R_ij(1, 1) + R_ij(2, 2))
    R23_R32 = (R_ij(1, 2) - R_ij(2, 1))
    R31_R13 = (R_ij(2, 0) - R_ij(0, 2))
    R12_R21 = (R_ij(0, 1) - R_ij(1, 0))
    # segundo renglon
    R11_R22_R33 = (R_ij(0, 0) - R_ij(1, 1) - R_ij(2, 2))
    R12R21 = (R_ij(0, 1) + R_ij(1, 0))
    R13R31 = (R_ij(0, 2) + R_ij(2, 0))
    # tercer renglon
    _R11R22_R33 = (-R_ij(0, 0) + R_ij(1, 1) - R_ij(2, 2))
    R23R32 = (R_ij(1, 2) + R_ij(2, 1))
    # cuarto renglon
    _R11_R22R33 = (-R_ij(0, 0) - R_ij(1, 1) + R_ij(2, 2))

    matriz_R = [
        [R11R22R33, R23_R32, R31_R13, R12_R21],
        [R23_R32, R11_R22_R33, R12R21, R13R31],
        [R31_R13, R12R21, _R11R22_R33, R23R32],
        [R12_R21, R13R31, R23R32, _R11_R22R33]
    ]
    return (matriz_R)

def rotation_matrix(matriz_R):
    """utilizando la funcion giant_matrix, fijando los valores de i,j
    se calcula la matriz de rotacion con los eigenvectores y eigenvalores
    arroja una matriz de rotacion que depende de la matriz gigante
    """
    eignvalues, eigenvectors = np.linalg.eig(matriz_R)
    q = eigenvectors[:, np.argmax(eignvalues)]
    q0, q1, q2, q3 = q[0], q[1], q[2], q[3]
    # matriz de rotacion con eigenvectores
    matriz_rotacion = np.array([
        [(q0 ** 2 + q1 ** 2 - q2 ** 2 - q3 ** 2), 2 * (q1 * q2 - q0 * q3), 2 * (q1 * q3 + q0 * q2)],
        [2 * (q1 * q2 + q0 * q3), (q0 ** 2 - q1 ** 2 + q2 ** 2 - q3 ** 2), 2 * (q2 * q3 - q0 * q1)],
        [2 * (q1 * q3 - q0 * q2), 2 * (q2 * q3 + q0 * q1), (q0 ** 2 - q1 ** 2 - q2 ** 2 + q3 ** 2)]
    ], dtype=np.float64)
    return (matriz_rotacion)

def rotation_vectors(vector_gorro, matriz_rotacion):
    """obtencion de vector rotado,
    utilizando la matriz de rotacion
    y los vectores gorro a rotar y trasladar"""

    coord_rotado = [np.matmul(
        matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    return (coord_rotado)

vecs1 = np.array([vecs for vecs in df_atoms1.vector.values])
vecs2 = np.array([vecs for vecs in df_atoms2.vector.values])

#baricentro
bari_1 = vecs1.mean(0)
bari_2 = vecs2.mean(0)

#vectores centricos
vecs_center_1 = vecs1 - bari_1
vecs_center_2 = vecs2 - bari_2

atoms_numbers1 = df_atoms1[df_atoms1.atom_name == 'CA'].index.values
atoms_numbers2 = df_atoms2[df_atoms2.atom_name == 'CA'].index.values

for idx in range(70):

    clique_in_protein_1 = df_rmsd_1[idx, 3]  # se extrae el indice de los cliques
    clique_in_protein_2 = df_rmsd_1[idx, 4]

    # se obtienen el numero de residuo de los candidatos
    los_del_clique_1 = df_new_df_clique1[clique_in_protein_1, [0, 1, 2, 3, 4, 5, 6]]

    los_del_clique_2 = df_new_df_clique2[clique_in_protein_2, [0, 1, 2, 3, 4, 5, 6]]

    parejas_cliques = [(i, j) for i, j in zip(los_del_clique_1, los_del_clique_2)]  # se crean las parejas de residuo de los cliques

    lista_vectores_gorro = []
    for bari in new_df_cliques1.baricentro_clique.values:
        lista_pre_vectores = [coord - bari for coord in df_atoms1[
            df_atoms1.atom_name == 'CA'][
            df_atoms1.residue_number.isin(los_del_clique_1) == False].vector.values]

        lista_vectores_gorro.append(lista_pre_vectores)

    vectores_gorro_proteina_1 = pd.DataFrame(lista_vectores_gorro)

    df_vectores_gorro_proteina_1 = vectores_gorro_proteina_1.values
    # OJO se extrae por indice no por numero de atomos!!!
    atoms1 = df_atoms1[df_atoms1.atom_name == 'CA'][df_atoms1.residue_number.isin(los_del_clique_1) == False].index.values
    atoms2 = df_atoms2[df_atoms2.atom_name == 'CA'][df_atoms2.residue_number.isin(los_del_clique_2) == False].index.values
    # valor para SO
    number_of_residues = df_atoms1.iloc[atoms1].shape[0]
    # se guardan los vectores
    protein_to_compare = np.array([vectors for vectors in df_atoms2.iloc[atoms2].vector.values])

    matriz_rotacion = df_rmsd_1[idx, 2]  # se obtiene la matriz de rotacion de los cliques candidatos

    # se obtienen los vectores gorro de la proteina a rotar y trasladar
    vector_gorro = df_vectores_gorro_proteina_1[clique_in_protein_1] # falta quitar los que no son del clique
    # se aplica la rotacion a vectores gorro
    coord_vectores_rotados = [np.matmul(matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    # se obtiene el baricentro de la proteina 2
    baricentro_proteina_2 = df_new_df_clique2[clique_in_protein_2, 22]
    # se suma el baricentro a vectores rotados de la proteina 1
    protein_trasladado_rotado = coord_vectores_rotados + baricentro_proteina_2  # nuevas coordendas proteina 1
    # se quitan residuos que pertenezcan al clique candidato para calcular la distancia y emparejar al mejor.

    protein_trasladado_rotado_sin_clique =protein_trasladado_rotado
    protein_to_compare_sin_clique = protein_to_compare

    residuos_match = []  # aqui se guardan las parejas de residuos

        # se itera por cada residuo ya rotado y trasladado

    for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado_sin_clique, atoms1):
        for residuo, res_2 in zip(protein_to_compare_sin_clique, atoms2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                residuos_match.append([distancia, (res_1, res_2)])

    residuos_match = sorted(residuos_match)
    
    c1 = []
    c2 = []
    cand_n = []
    for i in residuos_match:
        if (i[1][0] in c1) or (i[1][1] in c2) or (i[0] > 3.5):
            continue
        else:
            c1.append(i[1][0])
            c2.append(i[1][1])
            cand_n.append(i)
    
    
    parejas = [i[1] for i in cand_n]

    matriz_R = matrix_R()
    matriz_rotacion = rotation_matrix(matriz_R)
    vector_rotado = rotation_vectors(vecs_center_1, matriz_rotacion)
    vector_rotado_trasladado_a_clique2 = vector_rotado + np.array(bari_2)
    protein_trasladado_rotado = vector_rotado_trasladado_a_clique2
    protein_to_compare = vecs2
    
    residuos_match = []
    
    for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado, atoms_numbers1):
        for residuo, res_2 in zip(protein_to_compare, atoms_numbers2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                residuos_match.append([distancia, (res_1, res_2)])

#     residuos_match = sorted(residuos_match)

    c1 = []
    c2 = []
    cand_n = []
    for i in residuos_match:
        if (i[1][0] in c1) or (i[1][1] in c2) or (i[0] > 3.5):
            continue
        else:
            c1.append(i[1][0])
            c2.append(i[1][1])
            cand_n.append(i)

    df = pd.DataFrame(cand_n, columns=['distancia', 'candigatos'])
    so_temp = len(cand_n) / number_of_residues
    print(so_temp, idx)

0.046875 0
0.0 1
0.0 2
0.0 3
0.0 4
0.0 5
0.09375 6
0.234375 7
0.390625 8
0.390625 9
0.0 10
0.390625 11
0.375 12
0.09375 13
0.078125 14
0.078125 15
0.390625 16
0.0 17
0.5625 18
0.03125 19
0.0 20
0.0 21
0.0 22
0.0 23
0.03125 24
0.03125 25
0.0 26
0.0 27
0.0 28
0.0 29
0.015625 30
0.015625 31
0.34375 32
0.0 33
0.0 34
0.5 35
0.34375 36
0.0 37
0.03125 38
0.5 39
0.171875 40
0.359375 41
0.15625 42
0.0 43
0.0 44
0.171875 45
0.0 46
0.0 47
0.265625 48
0.0 49
0.0 50
0.0 51
0.5 52
0.609375 53
0.1875 54
0.0 55


KeyboardInterrupt: 

In [198]:
so_temp = len(cand_n) / number_of_residues

In [150]:
so_temp

0.03125

# Segundo intento ya con correccion de solo tomar carbonos alfa sin elementos del clique

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
sys.path.append('../math_tricks/')
import funciones_CLICK as fc
from scipy.spatial.distance import pdist, squareform
import multiprocessing
from functools import partial
import warnings
warnings.simplefilter("ignore")

%matplotlib inline

In [2]:
df_candidatos = pd.read_pickle('../df_alineados.pkl')

new_df_cliques1= pd.read_pickle('../clique1.pkl')
new_df_cliques2= pd.read_pickle('../clique2.pkl')

df_atoms1 = pd.read_pickle('../clique1_df_atributos.pkl')
df_atoms2 = pd.read_pickle('../clique2_df_atributos.pkl')

df_rmsd = pd.read_pickle('../df_rmsd.pkl')

In [3]:
df_candidatos.shape[0]

56

In [4]:
df_atoms1[df_atoms1.atom_name == 'CA'].shape[0]

71

In [6]:
56/(71)

0.7887323943661971

In [10]:
vecs1 = np.array([vecs for vecs in df_atoms1.vector.values])
vecs2 = np.array([vecs for vecs in df_atoms2.vector.values])

#baricentro
bari_1 = vecs1.mean(0)
bari_2 = vecs2.mean(0)

#vectores centricos
vecs_center_1 = vecs1 - bari_1
vecs_center_2 = vecs2 - bari_2

atoms_numbers1 = df_atoms1[df_atoms1.atom_name == 'CA'].index.values
atoms_numbers2 = df_atoms2[df_atoms2.atom_name == 'CA'].index.values

In [11]:
import read_pdb_tools as rpt
file1 = '../pdbs/1xxa_clean.pdb'
pdb1 = rpt.PdbStruct(file1)
pdb1.AddPdbData("%s" % file1)
pdb1.GetNeighbors()
pdb11 = pdb1.GetResChain()

In [12]:
parejas = df_candidatos.candigatos.values

def R_ij(i, j, parejas=parejas):
    valor = sum([vecs_center_1[pos_res[0]-1, i] * vecs_center_2[pos_res[1]-1, j] for pos_res in parejas])
    return valor

def matrix_R():
    """cliques a comparar: i,j
    desde aqui se itera sobre cada i y hay que variar los vectores
    coordenada
    Regresa la matriz gigante (matriz simetrica del articulo)"""
    # primer renglon
    R11R22R33 = (R_ij(0, 0) + R_ij(1, 1) + R_ij(2, 2))
    R23_R32 = (R_ij(1, 2) - R_ij(2, 1))
    R31_R13 = (R_ij(2, 0) - R_ij(0, 2))
    R12_R21 = (R_ij(0, 1) - R_ij(1, 0))
    # segundo renglon
    R11_R22_R33 = (R_ij(0, 0) - R_ij(1, 1) - R_ij(2, 2))
    R12R21 = (R_ij(0, 1) + R_ij(1, 0))
    R13R31 = (R_ij(0, 2) + R_ij(2, 0))
    # tercer renglon
    _R11R22_R33 = (-R_ij(0, 0) + R_ij(1, 1) - R_ij(2, 2))
    R23R32 = (R_ij(1, 2) + R_ij(2, 1))
    # cuarto renglon
    _R11_R22R33 = (-R_ij(0, 0) - R_ij(1, 1) + R_ij(2, 2))

    matriz_R = [
        [R11R22R33, R23_R32, R31_R13, R12_R21],
        [R23_R32, R11_R22_R33, R12R21, R13R31],
        [R31_R13, R12R21, _R11R22_R33, R23R32],
        [R12_R21, R13R31, R23R32, _R11_R22R33]
    ]
    return (matriz_R)

def rotation_matrix(matriz_R):
    """utilizando la funcion giant_matrix, fijando los valores de i,j
    se calcula la matriz de rotacion con los eigenvectores y eigenvalores
    arroja una matriz de rotacion que depende de la matriz gigante
    """
    eignvalues, eigenvectors = np.linalg.eig(matriz_R)
    q = eigenvectors[:, np.argmax(eignvalues)]
    q0, q1, q2, q3 = q[0], q[1], q[2], q[3]
    # matriz de rotacion con eigenvectores
    matriz_rotacion = np.array([
        [(q0 ** 2 + q1 ** 2 - q2 ** 2 - q3 ** 2), 2 * (q1 * q2 - q0 * q3), 2 * (q1 * q3 + q0 * q2)],
        [2 * (q1 * q2 + q0 * q3), (q0 ** 2 - q1 ** 2 + q2 ** 2 - q3 ** 2), 2 * (q2 * q3 - q0 * q1)],
        [2 * (q1 * q3 - q0 * q2), 2 * (q2 * q3 + q0 * q1), (q0 ** 2 - q1 ** 2 - q2 ** 2 + q3 ** 2)]
    ], dtype=np.float64)
    return (matriz_rotacion)

def rotation_vectors(vector_gorro, matriz_rotacion):
    """obtencion de vector rotado,
    utilizando la matriz de rotacion
    y los vectores gorro a rotar y trasladar"""

    coord_rotado = [np.matmul(
        matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    return (coord_rotado)

matriz_R = matrix_R()
matriz_rotacion = rotation_matrix(matriz_R)
vector_rotado = rotation_vectors(vecs_center_1, matriz_rotacion)
vector_rotado_trasladado_a_clique2 = vector_rotado + np.array(bari_2)
protein_trasladado_rotado = vector_rotado_trasladado_a_clique2
protein_to_compare = vecs2

new_df_atom1 = pd.concat([df_atoms1, pd.DataFrame(protein_trasladado_rotado, columns=['x', 'y', 'z'])], 1)
new_df_atom1['new_vector'] = [
    [new_df_atom1.iloc[i]['x'], new_df_atom1.iloc[i]['y'], new_df_atom1.iloc[i]['z']] for i in range(new_df_atom1.shape[0])]

for i in pdb11:
    mask = np.where(i.resi == new_df_atom1.residue_number, True, False)
    for j in new_df_atom1[mask].atom_name:
        mask_2 = np.where(new_df_atom1[mask].atom_name == j, True, False)
        i.GetAtom(j).UpDateValue('coord', new_df_atom1[mask][mask_2].new_vector.values[0])

pdb1.pdbdata = pdb11
pdb1.WriteToFile(file_out_name='1xxa_parejas_finales')